In [1]:
import glob
import os
import sys
import time
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import librosa
import torch
import torch.nn as nn
import torchaudio.functional as F
import torchaudio.transforms as T
import torch.optim as optim
import torchaudio
import IPython.display as ipd

import pickle5 as pickle
import tqdm
import IPython.display

from itertools import chain
from tqdm import tqdm
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from tabulate import tabulate
from IPython.display import Audio

from collections import defaultdict

from torch.autograd import Variable
from torch.utils.data import Dataset
from scipy.io.wavfile import read

from linformer_pytorch import Linformer
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import ConcatDataset, DataLoader, Dataset
from torchvision import datasets, transforms
from functools import partial

from ray import tune, air
from ray.tune import JupyterNotebookReporter
from ray.tune.schedulers import ASHAScheduler
from ray.air import session
from ray.air.checkpoint import Checkpoint

from tqdm.notebook import tqdm

from torch.nn.utils.rnn import pad_sequence

In [2]:
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"
torch.use_deterministic_algorithms(mode=True)

random_seed = 17 # or any of your favorite number

torch.manual_seed(random_seed)
torch.random.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

os.environ['PYTHONHASHSEED'] = str(random_seed)

In [3]:
!nvidia-smi

Sat Jun 24 11:52:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.182.03   Driver Version: 470.182.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   43C    P8    N/A /  N/A |    253MiB /  4042MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
class SpeechCommandsDataset(Dataset):
    """Google Speech Commands dataset."""

    def __init__(self, root_dir, split, transform=None):
        """
        Args:
            root_dir (string): Directory with all the data files.
            split    (string): In ["train", "valid", "test"].
        """
        self.root_dir = root_dir
        self.split = split
        self.transform = transform

        self.number_of_classes = len(self.get_classes())

        self.class_to_file = defaultdict(list)

        self.valid_filenames = self.get_valid_filenames()
        self.test_filenames = self.get_test_filenames()

        for c in self.get_classes():
            file_name_list = sorted(os.listdir(self.root_dir + "dataset/" + c))
            for filename in file_name_list:
                if split == "train":
                    if (filename not in self.valid_filenames[c]) and (filename not in self.test_filenames[c]):
                        self.class_to_file[c].append(filename)
                elif split == "valid":
                    if filename in self.valid_filenames[c]:
                        self.class_to_file[c].append(filename)
                elif split == "test":
                    if filename in self.test_filenames[c]:
                        self.class_to_file[c].append(filename)
                else:
                    raise ValueError("Invalid split name.")

        self.filepath_list = list()
        self.label_list = list()
        for cc, c in enumerate(self.get_classes()):
            f_extension = sorted(list(self.class_to_file[c]))
            l_extension = [cc for i in f_extension]
            f_extension = [self.root_dir + "dataset/" + c + "/" + filename for filename in f_extension]
            self.filepath_list.extend(f_extension)
            self.label_list.extend(l_extension)
        self.number_of_samples = len(self.filepath_list)
    def __len__(self):
        return self.number_of_samples

    def __getitem__(self, idx): 
        sample = np.zeros((16000, ), dtype=np.float32)

        sample_file = self.filepath_list[idx]

        sample_from_file = read(sample_file)[1]

        sample[:sample_from_file.size] = sample_from_file
        sample = sample.reshape((16000, ))

        # It is better to stick to PyTorch functions
        # I swapped the `librosa´ mfcc resampling with this PyTorch alternative
        # however, the parameters are the same
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        mfcc_transform = T.MFCC(
            sample_rate=16000,
            n_mfcc=20,
            melkwargs={
              'n_fft': 2048,
              'hop_length': 512})
        
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        sample = torch.transpose(mfcc_transform(torch.tensor(sample)), 0, 1)

        #sample = librosa.feature.mfcc(y=sample, sr=16000, hop_length=512, n_fft=2048).transpose().astype(np.float32)

        # we apply the augmentations on the sample if it is defined
        if self.transform:
            torch.manual_seed(random_seed)
            torch.cuda.manual_seed(random_seed)
            np.random.seed(random_seed)
            random.seed(random_seed)
            sample = self.transform(samples=sample.reshape(32, 1, 20), sample_rate=16000)
            sample = sample.reshape(32, 20)

        label = self.label_list[idx]

        return sample, label
    

    def get_classes(self):
        return ["zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine","right", 
                "left", "up", "down", "forward", "backward", "yes", "no", "stop", "start", "enable", 
                "disable", "ok", "cancel", "open", "close", "zoom in", "zoom out", "previous", "next", 
                "send", "receive", "move", "rotate", "record", "enter", "digit", "direction", "options", "undo"]

    def get_valid_filenames(self):
        class_names = self.get_classes()

        class_to_filename = defaultdict(set)
        with open(self.root_dir + "dataset/val.txt", "r") as fp:
            for line in fp:
                clean_line = line.strip().split("/")

                if clean_line[0] in class_names:
                    class_to_filename[clean_line[0]].add(clean_line[1])

        return class_to_filename

    def get_test_filenames(self):
        class_names = self.get_classes()

        class_to_filename = defaultdict(set)
        with open(self.root_dir + "dataset/test.txt", "r") as fp:
            for line in fp:
                clean_line = line.strip().split("/")

                if clean_line[0] in class_names:
                    class_to_filename[clean_line[0]].add(clean_line[1])

        return class_to_filename

In [6]:
from torch_audiomentations import Compose, Gain, PolarityInversion

# Initialize augmentation callable

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)
train_transforms = Compose(
    transforms=[
        Gain(
            min_gain_in_db=-15.0,
            max_gain_in_db=5.0,
            p=0.5,
        ),
        PolarityInversion(p=0.5)
    ]
)

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)
val_transforms = Compose(
    transforms=[
        Gain(
            min_gain_in_db=-15.0,
            max_gain_in_db=5.0,
            p=0.5,
        ),
        PolarityInversion(p=0.5)
    ]
)

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)
test_transforms = Compose(
    transforms=[
        Gain(
            min_gain_in_db=-15.0,
            max_gain_in_db=5.0,
            p=0.5,
        ),
        PolarityInversion(p=0.5)
    ]
)

In [7]:
dataset_folder= os.path.join(os.getcwd(), "Arabic_Speech_command/")

train_dataset = SpeechCommandsDataset(dataset_folder, "train")

valid_dataset = SpeechCommandsDataset(dataset_folder, "valid")

test_dataset  = SpeechCommandsDataset(dataset_folder, "test")

### Augmented Datasets ###
# Note we should concatenate both the non-augmented and the augmented training datasets

train_dataset_aug = ConcatDataset([SpeechCommandsDataset(dataset_folder, "train", 
                                                         transform=train_transforms),
                                                         train_dataset])

valid_dataset_aug = SpeechCommandsDataset(dataset_folder, "valid", transform=val_transforms)

test_dataset_aug  = SpeechCommandsDataset(dataset_folder, "test", transform=test_transforms)

In [8]:
print(f"Training dataset, {len(train_dataset)} samples.")

print(f"Validation dataset, {len(valid_dataset)} samples.")

print(f"Testing dataset, {len(test_dataset)} samples.")

Training dataset, 7201 samples.
Validation dataset, 2399 samples.
Testing dataset, 2400 samples.


In [9]:
class NeuralNetworkModel(nn.Module):
    """Neural network model (Transformer-based).

    Args:
        idim (int): Input feature dimension.
        d_att (int): Attention dimension.
        n_heads (int): The number of attention heads.
        d_ff (int): Dimension of feed forward network.
        dropout_rate (float): Dropout rate.
        n_layers (int): The number of encoder layers.
        d_linear (int): Dimension of a hidden layer of the classifier.
        n_classes (int): The number of the output classes.

    """

    def __init__(
        self,
        idim=13,
        d_att=64,
        n_heads=4,
        d_ff=512,
        dropout_rate=0.1,
        n_layers=3,
        n_classes=40
    ):
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        super().__init__()
        self.subsampling = Subsampling(idim=idim, d_att=d_att)
        self.positional_encoding = PositionalEncoding()
        
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        self.encoder_layers = nn.Sequential()
        for i in range(n_layers):
            self.encoder_layers.add_module(
                f'EncoderLayer{i}', 
                TransformerEncoderLayer(d_att, n_heads, d_ff, dropout_rate)
            )
            
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        self.norm = nn.LayerNorm(d_att)#, elementwise_affine=False)
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        self.dropout = nn.Dropout(dropout_rate)
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        self.out = nn.Linear(d_att, n_classes)#, bias=False)

    def forward(self, x):
        """Recognize the input speech commands.

        Args:
            x (torch.Tensor): Input features (batch, tmax, idim).

        Returns:
            torch.Tensor: Recognized classes (batch, num_classes).

        """   
        # Transformer encoder
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        x = self.subsampling(x)
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        x = self.positional_encoding(x)
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        x = self.encoder_layers(x)
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        x = self.norm(x)
        # Classifier
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        x = torch.mean(x, dim=1)  # (b, t, d_att) -> (b, d_att)
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        x = self.dropout(x)
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        x = self.out(x)

        return x

class TransformerEncoderLayer(nn.Module):
    """A Transformer encoder layer.

    Args:
        d_att (int): Attention dimension.
        d_head (int): The number of attention heads.
        d_ff (int): Dimension of feed forward network.
        dropout_rate (float): Dropout rate.

    """

    def __init__(self, d_att, d_head, d_ff, dropout_rate=0.1):
        super().__init__()
        
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        self.mha = MultiHeadAttention(d_att, d_head, dropout_rate)
        
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        self.ff = FeedForward(d_att, d_ff, dropout_rate)
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        self.norm_mha = nn.LayerNorm(d_att)#, elementwise_affine=False)
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        self.norm_ff = nn.LayerNorm(d_att)#, elementwise_affine=False)
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self, x):
        """

        Args:
            x (torch.Tensor): Pre-encoded inputs (batch, tmax, d_att).

        Returns:
            torch.Tensor: Encoded outputs (batch, tmax, d_att).

        """
        # Multi-Head Attention
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        res = x
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        x = self.norm_mha(x)
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        x = res + self.dropout(self.mha(x, x, x))

        # Feed-Forward
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        res = x
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        x = self.norm_ff(x)
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        x = res + self.dropout(self.ff(x))

        return x
class Subsampling(nn.Module):
    """Convolutional Subsampling.

    Args:
        idim (int): Input feature dimension.
        d_att (int): Attention dimension.

    """

    def __init__(self, idim, d_att):
        super().__init__()
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        self.conv = nn.Sequential(
            nn.Conv2d(1, d_att, kernel_size=(3, 3), stride=(2, 2)),#, bias=False),
            nn.ReLU(),
            nn.Conv2d(d_att, d_att, kernel_size=(3, 3), stride=(2, 2)),#, bias=False),
            nn.ReLU(),
        )
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        self.linear = nn.Linear(256, d_att)#, bias=False)

    def forward(self, x):
        """

        Args:
            x (torch.Tensor): Input features (batch, tmax, idim).

        Returns:
            torch.Tensor: Subsampled features (batch, tmax', d_att).

        """
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        x = x.unsqueeze(1)  # (b, t, idim) -> (b, c=1, t, idim)
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        x = self.conv(x)
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        b, c, t, f = x.size()
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        #x = x.view(x.size(0), -1)
        x = x.transpose(1, 2).contiguous().view(b, t, c * f)  # (b, c, t, f) -> (b, t, c * t)
        
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        x = self.linear(x)

        return x

class PositionalEncoding(nn.Module):
    """Positional Encoding.

    Args:
        idim (int): Input feature dimension.

    """

    def __init__(self):
           super().__init__()
        
    def forward(self, x):
        """

        Args:
            x: (torch.Tensor): Subsampled features (batch, tmax, d_att).

        Returns:
            torch.Tensor: Encoded features (batch, tmax, d_att).

        """
        
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        _, tmax, d_att = x.size()
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        pos = torch.arange(0, tmax, dtype=torch.float32).unsqueeze(1)
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        pe = torch.zeros(1, tmax, d_att, dtype=torch.float32).to(x.device)
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        pe[:, :, 0::2] = torch.sin(pos / torch.pow(10000, torch.arange(0, d_att, 2) / d_att))
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        pe[:, :, 1::2] = torch.cos(pos / torch.pow(10000, torch.arange(0, d_att, 2) / d_att))
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        x = x + pe

        return x

class MultiHeadAttention(nn.Module):
    """Multi-Head Attention.

    Args:
        d_att (int): Dimension of attention.
        d_head (int): The number of attention heads.
        dropout_rate (float): Dropout rate.

    """

    def __init__(self, d_att, n_heads, dropout_rate):
        super().__init__()
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        self.linear_q = nn.Linear(d_att, d_att)#, bias=False)
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        self.linear_k = nn.Linear(d_att, d_att)#, bias=False)
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        self.linear_v = nn.Linear(d_att, d_att)#, bias=False)
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        self.linear_head = nn.Linear(d_att, d_att)#, bias=False)
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        self.dropout = nn.Dropout(dropout_rate)
        
        self.n_heads = n_heads
        self.d_comn = d_att // self.n_heads

    def forward(self, q, k, v):
        """

        Args:
            q: (torch.Tensor): Query (batch, tmax, d_att).
            k: (torch.Tensor): Key (batch, tmax, d_att).
            v: (torch.Tensor): Value (batch, tmax, d_att).
            Returns:
            torch.Tensor: Output shape (batch, tmax, d_att).

        """
        # Linear
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        qw = self.linear_q(q)
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        kw = self.linear_k(k)
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        vw = self.linear_v(v)

        # Reshape tensor (b, t, d_att) -> (b, n_heads, t, d_comn)
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        b, t, d_att = q.size()
        
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        qw = qw.view(b, t, self.n_heads, self.d_comn).transpose(1, 2)
        
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        kw = kw.view(b, t, self.n_heads, self.d_comn).transpose(1, 2)
        
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        vw = vw.view(b, t, self.n_heads, self.d_comn).transpose(1, 2)

        # Dot-attention
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        matmul = torch.matmul(qw, kw.transpose(2, 3))
        
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        scale = matmul / torch.sqrt(torch.tensor(self.d_comn))
        
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        softmax = torch.softmax(scale, dim=-1)
        
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        att = torch.matmul(self.dropout(softmax), vw)  # (b, n_heads, t, d_comn)

        # Concatenate
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        att = att.transpose(1, 2).contiguous().view(b, -1, self.n_heads * self.d_comn)  # (b, t, d_att)

        # Linear
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        mha = self.linear_head(att)

        return mha

class FeedForward(nn.Module):
    """Feed-Forward Network.

    Args:
        d_ff (int): Dimension of feed-forward network.

    """

    def __init__(self, d_att, d_ff, dropout_rate):
        super().__init__()
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        self.ff = nn.Sequential(
            nn.Linear(d_att, d_ff),#, bias=False), 
            nn.ReLU(), 
            nn.Dropout(dropout_rate), 
            nn.Linear(d_ff, d_att)#, bias=False)
        )

    def forward(self, x):
        """

        Args:
            x: (torch.Tensor): Input shape (batch, tmax, d_att).

        Returns:
            torch.Tensor: Output shape (batch, tmax, d_att).

        """
        torch.manual_seed(random_seed)
        torch.cuda.manual_seed(random_seed)
        np.random.seed(random_seed)
        random.seed(random_seed)
        return self.ff(x)

In [10]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(random_seed)

In [11]:
# define some metric lists
def train(config, 
          train_set, 
          val_set, 
          results_dir='./results',
          continue_training=False,
          epochs=5, 
          plot=False):
    
    
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    np.random.seed(random_seed)
    random.seed(random_seed)
    model = NeuralNetworkModel(n_heads=config["n_heads"], 
                               n_layers=config["n_layers"]
                               ).to(device)
    
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    np.random.seed(random_seed)
    random.seed(random_seed)
    criterion = nn.CrossEntropyLoss()
    
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    np.random.seed(random_seed)
    random.seed(random_seed)
    optimizer = optim.Adam(model.parameters(), lr=config["lr"])
    
    if continue_training:
        model_state, optimizer_state = torch.load(
            os.path.join(results_dir, "checkpoint"))
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)
        
        
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    np.random.seed(random_seed)
    random.seed(random_seed)
    train_loader = DataLoader(
        train_set,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=1,
        worker_init_fn=seed_worker)
    
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    np.random.seed(random_seed)
    random.seed(random_seed)
    valid_loader = DataLoader(
        val_set,
        batch_size=int(config["batch_size"]),
        shuffle=False,
        num_workers=1,
        worker_init_fn=seed_worker)
    

    #store evaluation metrics in these lists
    train_acc_list = []
    train_loss_list = []
    dev_acc_list = []
    dev_loss_list = []
    
    # best loss tracking for model checkpointing
    best_loss = float("inf")

    # Train the model
    for epoch in range(epochs):

        batch_train_loss = 0
        batch_train_acc = 0
        batch_dev_loss = 0
        batch_dev_acc = 0

        # Training
        model.train()
        for feats, labels in train_loader:
            feats, labels = feats.to(device), labels.to(device)

            # Reset gradients
            optimizer.zero_grad()

            # Forward
            outputs = model(feats)
            loss = criterion(outputs, labels)

            # Backward
            loss.backward()

            # Update weights
            optimizer.step()

            batch_train_loss += loss.item()
            batch_train_acc += (outputs.max(1)[1] == labels).sum().item()

        train_loss_list.append(batch_train_loss / len(train_loader))
        train_acc_list.append(batch_train_acc / len(train_loader.dataset))
        # Validation
        
        model.eval()
        with torch.no_grad():
            for feats, labels in valid_loader:
                feats, labels = feats.to(device), labels.to(device)

                # Forward
                outputs = model(feats)
                loss = criterion(outputs, labels)

                batch_dev_loss += loss.item()
                batch_dev_acc += (outputs.max(1)[1] == labels).sum().item()

        dev_loss_list.append(batch_dev_loss / len(valid_loader))
        dev_acc_list.append(batch_dev_acc / len(valid_loader.dataset))
        
        os.makedirs(results_dir, exist_ok=True)
        torch.save((model.state_dict(), optimizer.state_dict()), os.path.join(results_dir, f"checkpoint{epoch}.pt"))
        
        checkpoint = Checkpoint.from_directory(results_dir)
        session.report({
                        "loss" : dev_loss_list[-1],
                        "accuracy" : dev_acc_list[-1]
                        },
                        checkpoint=checkpoint)
        
        # plot results if flag is True
        if plot:
            fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 6.5))

            ax[0].plot(range(len(train_loss_list)), train_loss_list, label='train loss')
            ax[0].plot(range(len(dev_loss_list)), dev_loss_list, label='val loss')
            ax[0].set_title('Loss Evolution')

            ax[1].plot(range(len(train_acc_list)), train_acc_list, label='train accuracy')
            ax[1].plot(range(len(dev_acc_list)), dev_acc_list, label='val accuracy')
            ax[1].set_title('Accuracy Evolution')

            ax[0].legend()
            ax[1].legend()

            plt.show()
        
def test(model, test_set, device, batch_size=64):
    best_loss = float("inf")
    
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    np.random.seed(random_seed)
    random.seed(random_seed)
    test_loader = DataLoader(
        test_set, batch_size=batch_size, shuffle=False, num_workers=1)

    # Test
    test_start_time = time.time()
    
    batch_test_loss = 0
    batch_test_acc = 0

    with torch.no_grad():
        for feats, labels in test_loader:
            feats, labels = feats.to(device), labels.to(device)

            # Forward
            outputs = model(feats)
            batch_test_acc += (outputs.max(1)[1] == labels).sum().item()

    test_mean_acc = batch_test_acc / len(test_loader.dataset)

    # Print out test time
    test_time = time.time() - test_start_time
    print(f">> Test Time: {test_time}")
    
    return test_mean_acc

# Train/Validation Runs

In [ ]:
num_samples=500
max_num_epochs=100

config = {
    "n_heads": tune.choice([8, 16]),
    "n_layers": tune.choice([10, 11, 12, 13]),
    "lr": tune.loguniform(1e-5, 1e-4),
    "batch_size": tune.choice([16, 32, 64, 128])
}

scheduler = ASHAScheduler(
        metric="accuracy",
        mode="max",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)

reporter = JupyterNotebookReporter(
        overwrite=True,
        metric_columns=["loss", "accuracy", "training_iteration"])

tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(
                    partial(train, 
                        train_set=train_dataset, 
                        val_set=valid_dataset,
                        epochs=max_num_epochs)),
            resources={"cpu": 8, "gpu": 1}
        ),
        tune_config=tune.TuneConfig(
            scheduler=scheduler,
            num_samples=num_samples,
        ),
        param_space=config,
        run_config=air.RunConfig(local_dir="/media/mohamed/FCF05CB8F05C7B3A/ray_results",
                                 progress_reporter=reporter),
    )

results = tuner.fit()

best_trial = results.get_best_result("accuracy", "max", "last")
print(f"Best trial config: {best_trial.config}")

2023-06-24 11:53:31,074	INFO worker.py:1550 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Trial name,accuracy,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
tune_with_parameters_fb485_00000,0.522718,2023-06-24_11-57-34,False,,1faf94f2d19147c189496039a19d5ef4,mbe19,14,1.9866,139.174.120.39,13436,True,238.094,17.7701,238.094,1687600654,0,,14,fb485_00000,0.00302267


In [38]:
num_samples=1
max_num_epochs=5

config = {
    "n_heads": 4,#tune.choice([4, 8, 16]),
    "n_layers": 8,#tune.choice([9, 10, 11, 12, 13, 14]),
    "lr": 1e-5,#tune.loguniform(1e-5, 1e-4),
    "batch_size": 64#tune.choice([32, 64, 128])
}

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)
scheduler = ASHAScheduler(
        metric="accuracy",
        mode="max",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)

reporter = JupyterNotebookReporter(
        overwrite=True,
        metric_columns=["loss", "accuracy", "training_iteration"])

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)
tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(
                    partial(train, 
                        train_set=train_dataset, 
                        val_set=valid_dataset,
                        epochs=max_num_epochs)),
            resources={"cpu": 8, "gpu": 1}
        ),
        tune_config=tune.TuneConfig(
            scheduler=scheduler,
            num_samples=num_samples
        ),
        param_space=config,
        run_config=air.RunConfig(local_dir="/media/mohamed/FCF05CB8F05C7B3A/ray_results",
                                 progress_reporter=reporter),
    )

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)
results = tuner.fit()

best_trial = results.get_best_result("accuracy", "max", "last")
print(f"Best trial config: {best_trial.config}")

Trial name,accuracy,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
tune_with_parameters_44d8d_00000,0.0854523,2023-05-08_13-28-34,True,,bee6f4a09a254e29ac2b80958fad9e93,mbe19,5,3.58259,139.174.120.27,7683,True,82.9564,16.3727,82.9564,1683545314,0,,5,44d8d_00000,0.0028758


2023-05-08 13:28:34,530	INFO tune.py:799 -- Total run time: 85.63 seconds (85.57 seconds for the tuning loop).


Best trial config: {'n_heads': 4, 'n_layers': 8, 'lr': 1e-05, 'batch_size': 64}


In [15]:
from pprint import pprint

pprint(best_trial)

Result(metrics={'loss': 1.0552736018833362, 'accuracy': 0.8315964985410588, 'should_checkpoint': True, 'done': True, 'trial_id': '6a021_00350', 'experiment_tag': '350_batch_size=64,dropout_rate=0.1000,lr=0.0008,n_heads=16,n_layers=5'}, error=None, log_dir=PosixPath('/media/mohamed/FCF05CB8F05C7B3A/ray_results/tune_with_parameters_2023-03-26_13-28-59/tune_with_parameters_6a021_00350_350_batch_size=64,dropout_rate=0.1000,lr=0.0008,n_heads=16,n_layers=5_2023-03-26_18-06-36'))


In [ ]:
best_trial = result.get_best_trial("loss", "min", "last")

best_trained_model = NeuralNetworkModel(dropout_rate=best_trial.config["dropout_rate"], 
                                        n_heads=best_trial.config["n_heads"])

best_trained_model.to(device)

best_checkpoint = result.get_best_checkpoint(trial=best_trial, metric="loss", mode="min")
best_checkpoint_dir = best_checkpoint.to_directory(path="results")

model_state, optimizer_state = torch.load(os.path.join(best_checkpoint_dir, "checkpoint.pt"))
best_trained_model.load_state_dict(model_state)

test_acc = test(model=best_trained_model, test_set=test_dataset,
                device=device, batch_size=best_trial.config["batch_size"])

print(f"Best trial test set accuracy: {test_acc}")

In [ ]:
## Train Data (augmented) with Validation Data (Augmented)
# We can notice that the max accuracy is reaching a value of ~ 77%

# Build a model
model = NeuralNetworkModel(dropout_rate=best_trial.config["dropout_rate"], 
                               n_heads=best_trial.config["n_heads"]).to(device)

#best_trial.config["lr"]
#best_trial.config["batch_size"]
        
print(f"# model parameters: {sum(p.numel() for p in aug_model.parameters()):,}")

# Define an optimizer and a loss function
optimizer = optim.Adam(aug_model.parameters(), lr=best_trial.config["lr"])
loss_fn = nn.CrossEntropyLoss()

epochs = 100

train(, optimizer=optimizer, 
      train_loader=train_loader_aug, valid_loader=valid_loader_aug, 
      criterion=aug_loss_fn, epochs=epochs, plot=True)

# Test Runs

In [ ]:
## Testing the model with the test augmented data

test(model=aug_model, test_loader=test_loader_aug, criterion=aug_loss_fn)

In [ ]:
## Testing the model with the test augmented data

test(model=baseline_model, test_loader=test_loader_aug, criterion=baseline_loss_fn)

In [ ]:
## Testing the model with the test non-augmented data

test(model=aug_model, test_loader=test_loader, criterion=aug_loss_fn)

In [ ]:
## Testing the model with the test non-augmented data

test(model=baseline_model, test_loader=test_loader, criterion=baseline_loss_fn)